In [240]:
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [241]:
def preprocess_image(image):
    image = tf.cond(
        tf.image.is_jpeg(image),
        lambda: tf.image.decode_jpeg(image, channels=3),
        lambda: tf.image.decode_png(image, channels=3))
    image = tf.image.resize(image, (56, 56))
    image = (image - 127.5) / 127.5  # [-1, 1]

    return image

In [242]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [243]:
def load_normalized_dataset(path):

    image_samples_path = list()
    class_count = 1
    max_samples = -1
    max_classes = -1
    for class_dir in os.listdir(path):
        img_sample_count = 1
        full_class_dir = os.path.join(path, class_dir)
        for image_name in os.listdir(full_class_dir):
            
            full_image_name = os.path.join(full_class_dir, image_name)
            image_samples_path.append(full_image_name)
            
            if img_sample_count == max_samples:
                break

            img_sample_count += 1

        if class_count == max_classes:
            break

        class_count += 1

    random.shuffle(image_samples_path)

    with open('image_list.txt', 'w') as f:
        for item in image_samples_path:
            f.write("%s\n" % item)

    scene_dataset = tf.data.Dataset.from_tensor_slices(image_samples_path)
    scene_dataset = scene_dataset.map(load_and_preprocess_image)
    batch_size = 256
    scene_dataset = scene_dataset.batch(batch_size)

    return scene_dataset

In [244]:
os.path.abspath("")

'F:\\SKOLA\\FAKS\\RI-Logo-generation-GAN\\src'

In [245]:
try:
    f = open("../data/Amazon/Amazon logo2.jpg")
except FileNotFoundError:
    print('Error')
else:
    print('Dobar')


Dobar


In [246]:
scene_dataset = load_normalized_dataset("../data")
scene_dataset

<_BatchDataset element_spec=TensorSpec(shape=(None, 56, 56, 3), dtype=tf.float32, name=None)>

In [247]:
def generator():
    model = keras.Sequential([
        layers.Dense(units=7 * 7 * 256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),

        layers.Conv2DTranspose(filters=128, kernel_size=(5, 5), strides=(1, 1), padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(filters=256, kernel_size=(5, 5), strides=(2, 2), padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(filters=32, kernel_size=(5, 5), strides=(2, 2), padding="same", use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(filters=3, kernel_size=(5, 5), strides=(2, 2), padding="same", use_bias=False,
                               activation="tanh"),
    ])
    return model

In [248]:
 gen_model = generator()

In [249]:
def generator_loss(fake_output):
    cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [250]:
def generator_optimizer():
    return tf.optimizers.Adam(1e-4)

In [251]:
gen_optimizer = generator_optimizer()

In [252]:
def discriminator():
    model = keras.Sequential([
        layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), padding='same',
                      input_shape=[56, 56, 3]),
        layers.LeakyReLU(),
        layers.Dropout(rate=0.3),

        layers.Conv2D(filters=128, kernel_size=(5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(rate=0.3),

        layers.Flatten(),
        layers.Dense(units=1),
    ])
    return model

In [253]:
dis_model = discriminator()

In [254]:
def discriminator_loss(real_output,
                       fake_output):
    cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss

    return total_loss

In [255]:
def discriminator_optimizer():
    return tf.optimizers.Adam(1e-4)

In [256]:
dis_optimizer = discriminator_optimizer()

In [257]:
def train_step(images, gen_model, gen_optimizer, dis_model, dis_optimizer, gen_loss_metric, dis_loss_metric):
    noise = tf.random.normal([256, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = gen_model(noise, training=True)

        real_output = dis_model(images, training=True)
        fake_output = dis_model(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, dis_model.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, gen_model.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_discriminator, dis_model.trainable_variables))

    gen_loss_metric(gen_loss)
    dis_loss_metric(disc_loss)



In [258]:
def test_step(real_images, gen_model, dis_model):
    random_seed = tf.random.normal([256, 100])
    fake_images = gen_model(random_seed, training=False)

    real_dis_prediction = dis_model(real_images)
    fake_dis_prediction = dis_model(fake_images)

    correct = len(real_dis_prediction[real_dis_prediction >= 0.0])
    wrong = len(real_dis_prediction[real_dis_prediction < 0.0])
    real_dis_acc = float(correct) / float(correct + wrong)

    correct = len(fake_dis_prediction[fake_dis_prediction < 0.0])
    wrong = len(fake_dis_prediction[fake_dis_prediction >= 0.0])
    fake_dis_acc = float(correct) / float(correct + wrong)

    combined_dis_acc = (real_dis_acc + fake_dis_acc) / 2

    return real_dis_acc, fake_dis_acc, combined_dis_acc

In [259]:
def loss_and_accuracy(train_summary_writer, gen_loss_metric, epoch, dis_loss_metric, real_dis_acc, fake_dis_acc, combined_dis_acc):
    with train_summary_writer.as_default():
        with tf.name_scope('Loss'):
            tf.summary.scalar('Generator', gen_loss_metric.result(), step=epoch)
            tf.summary.scalar('Discriminator', dis_loss_metric.result(), step=epoch)

        with tf.name_scope('Accuracy'):
            tf.summary.scalar('Real Discriminator', real_dis_acc, step=epoch)
            tf.summary.scalar('Fake Discriminator', fake_dis_acc, step=epoch)
            tf.summary.scalar('Combined Discriminator', combined_dis_acc, step=epoch)

    gen_loss_metric.reset_states()
    dis_loss_metric.reset_states()

In [260]:
def weights_and_biases(train_summary_writer, gen_model, epoch, dis_model):
    with train_summary_writer.as_default():
        with tf.name_scope('Generator'):
            with tf.name_scope('Weights'):
                for gen_layer in gen_model.layers:
                    try:
                        tf.summary.histogram(gen_layer.name, gen_layer.get_weights()[0], step=epoch)
                    except Exception:
                        pass

        with tf.name_scope('Discriminator'):
            for dis_layer in dis_model.layers:
                try:
                    with tf.name_scope('Weights'):
                        tf.summary.histogram(dis_layer.name, dis_layer.get_weights()[0], step=epoch)
                    with tf.name_scope('Biases'):
                        tf.summary.histogram(dis_layer.name, dis_layer.get_weights()[1], step=epoch)
                except Exception:
                    pass

In [261]:
def save_image(gen_model, epoch, train_summary_writer):
    test_input = tf.random.normal([16, 100])
    predictions = None
  #  if (epoch % 100 == 0):
    predictions = gen_model(test_input, training=False)

   # if epoch % 100 == 0:
    for i in range(predictions.shape[0]):
        img = np.array(predictions[i]) * 127.5 + 127.5
        img = img.astype(np.uint8, copy=False)
        plt.imsave('../res/image_at_epoch_{:05d}_{:05d}.png'.format(epoch, i), img)

   # if epoch % 100 == 0:
    with tf.name_scope("Samples"):
        with train_summary_writer.as_default():
            for i in range(predictions.shape[0]):
                img = np.array(np.array(predictions[i]) * 0.5 + 0.5).astype(np.float32)
                img = np.reshape(img, (1, 56, 56, 3))
                tf.summary.image('image_{:03d}.png'.format(i), img, step=epoch)

In [262]:
def train(real_image_dataset, last_epoch, epochs, gen_model, gen_optimizer, dis_model, dis_optimizer):
    gen_loss_metric = keras.metrics.Mean('train_loss', dtype=tf.float32)
    dis_loss_metric = keras.metrics.Mean('train_loss', dtype=tf.float32)
    train_log_dir = '../log'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    for epoch in range(last_epoch, epochs):
        for image_batch in real_image_dataset:
            train_step(image_batch, gen_model, gen_optimizer, dis_model, dis_optimizer, gen_loss_metric, dis_loss_metric)
            real_dis_acc, fake_dis_acc, combined_dis_acc = test_step(image_batch, gen_model, dis_model)
            loss_and_accuracy(train_summary_writer, gen_loss_metric, epoch, dis_loss_metric, real_dis_acc, fake_dis_acc, combined_dis_acc)
            weights_and_biases(train_summary_writer, gen_model, epoch, dis_model)
        print("Generisem sliku")
        save_image(gen_model, epoch + 1, train_summary_writer)

In [ ]:
train(real_image_dataset=scene_dataset, last_epoch=0,epochs=100,gen_model=gen_model, gen_optimizer=gen_optimizer, dis_model=dis_model,
            dis_optimizer=dis_optimizer)

Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
Generisem sliku
